In [1]:
import pandas as pd
import numpy as np

In [36]:
def round_hour(row, variable):
    if int(row[variable + '_min']) > 30:
        return int(row[variable +'_hour']) + 1
    else:
        return int(row[variable +'_hour'])

In [35]:
def get_cyclical_time(variable):
    df[variable]= df[variable].replace([2400],0)
    df[variable] = df[variable].astype(int).astype(str).str.zfill(4) 
    df[variable] = pd.to_datetime(df[variable], format='%H%M')
    df[variable +'_hour'] = df[variable].dt.hour
    df[variable + '_min'] = df[variable].dt.minute
    df[variable +'_hour'] = df.apply(lambda x: round_hour(x, variable), axis=1)
    df['sin_' + variable] = np.sin(df[variable +'_hour'])
    df['cos_' + variable] = np.cos(df[variable +'_hour'])

In [42]:
def is_delayed(row):
    if row['arr_delay'] <= 15:
        return 0
    else:
        return 1

def how_delayed(row):
    if row['arr_delay'] <= 15:
        return 0
    else:
        return row['arr_delay']

In [1]:
def get_features(path, tvt, csv):
    df = pd.read_csv(path + csv)
    df['fl_date'] = pd.to_datetime(df['fl_date'])
    df['date']= df['fl_date'].dt.day
    df['weekday'] = df['fl_date'].dt.dayofweek
    df['sin_weekday'] = np.sin(df['weekday'])
    df['cos_weekday'] = np.cos(df['weekday'])
    get_cyclical_time('crs_dep_time')
    get_cyclical_time('dep_time')
    get_cyclical_time('crs_arr_time')
    get_cyclical_time('arr_time')
    df['delayed'] = df.apply(lambda x: is_delayed(x), axis=1)
    df['delay'] = df.apply(lambda x: how_delayed(x), axis=1)
    # average departure delay and number of fligths per carrier
    df = df.join(df.groupby('op_unique_carrier').mean()['dep_delay'], on='op_unique_carrier', rsuffix='carrier_dep_delay')
    df = df.join(df.groupby('op_unique_carrier').count()['fl_date'], on='op_unique_carrier', rsuffix='carrier_flight_count')
    # average departure delay and number of flights per origin
    df = df.join(df.groupby('origin').mean()['dep_delay'], on='origin', rsuffix='origin_dep_delay')
    df = df.join(df.groupby('origin').count()['fl_date'], on='origin', rsuffix='origin_flight_count')
    # average departure delay and number of flights per destination
    df = df.join(df.groupby('dest').mean()['dep_delay'], on='dest', rsuffix='dest_dep_delay')
    df = df.join(df.groupby('dest').count()['fl_date'], on='dest', rsuffix='dest_flight_count')
    # average arrival delay per carrier, origin and destination
    df = df.join(df.groupby('op_unique_carrier').mean()['arr_delay'], on='op_unique_carrier', rsuffix='carrier_arr_delay')
    df = df.join(df.groupby('origin').mean()['arr_delay'], on='origin', rsuffix='origin_arr_delay')
    df = df.join(df.groupby('dest').mean()['arr_delay'], on='dest', rsuffix='dest_arr_delay')
    # average arrival delay and number of flights per hour(Dep)
    df = df.join(df.groupby('crs_dep_time_hour').mean()['arr_delay'], on='crs_dep_time_hour', rsuffix='hourly_arr_delay')
    df = df.join(df.groupby('crs_dep_time_hour').count()['arr_delay'], on='crs_dep_time_hour', rsuffix='hourly_flight_count')
    # average departure delay and number of flights per hour(Dep)
    df = df.join(df.groupby('crs_dep_time_hour').mean()['dep_delay'], on='crs_dep_time_hour', rsuffix='hourly_dep_delay')
    # average arrival delay and number of flights per hour (per origin and per carrier)
    df = df.join(df.groupby(['crs_hour_dep', 'origin']).mean()['arr_delay'], on=['crs_hour_dep', 'origin'], rsuffix='dephourly_arrdelay_origin')
    df = df.join(df.groupby(['crs_hour_dep', 'op_unique_carrier']).mean()['arr_delay'], on=['crs_hour_dep', 'op_unique_carrier'], rsuffix='dephourly_arrdelay_carrier')
    # average arrival delay and number of flights per hour (per origin and per carrier)
    df = df.join(df.groupby(['crs_hour_dep', 'origin']).mean()['dep_delay'], on=['crs_hour_dep', 'origin'], rsuffix='dephourly_depdelay_origin')
    df = df.join(df.groupby(['crs_hour_dep', 'op_unique_carrier']).mean()['dep_delay'], on=['crs_hour_dep', 'op_unique_carrier'], rsuffix='dephourly_depdelay_carrier')
    df.to_csv(f'{path}engineered_{csv}', index=None)